In [1]:
import pandas as pd
import numpy as np

dates=pd.date_range('2017',periods = 16, freq = 'Q')
sigma, mu = 1, 0
dataseries=pd.Series(sigma * np.random.randn(16) + mu,index=dates)
df=pd.DataFrame({'Test':dataseries})
df.head()

,Test
2017-03-31,0.361663
2017-06-30,-1.706275
2017-09-30,-0.343944
2017-12-31,-0.465031
2018-03-31,-1.331470


In [2]:
s1 = df['Test']

In [3]:
def calculate_ratio(ts1, ts2):
    '''
    '''
    return s1 / s2


def calculate_difference(ts):
    '''
    '''
    return ts - ts.shift(1)

def calculate_transform(ts):
    '''
    '''
    return ts / ts.shift(1) - 1

def calculate_inverse_difference(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] + ts.iloc[i]
    return pd.Series(vals, ts.index)

def calculate_inverse_transform(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] * (1 + ts.iloc[i])
    return pd.Series(vals, ts.index)


def calculate_standarization(ts, mu, sigma): 
    '''
    '''
    return (ts - mu) / sigma  

In [4]:
s2 = calculate_difference(s1)
start = s1.iloc[0]
s3 = calculate_inverse_difference(s2, start)
s4 = calculate_transform(s1)
s5 = calculate_inverse_transform(s4, start)
s6 = calculate_standarization(s1, mu, 0)

In [5]:
s1

2017-03-31    0.361663
2017-06-30   -1.706275
2017-09-30   -0.343944
2017-12-31   -0.465031
2018-03-31   -1.331470
2018-06-30   -0.828993
2018-09-30    1.156199
2018-12-31   -0.759970
2019-03-31    1.684245
2019-06-30    0.436408
2019-09-30    1.612366
2019-12-31   -0.972930
2020-03-31   -0.624794
2020-06-30   -1.621668
2020-09-30   -0.524856
2020-12-31   -0.709843
Freq: Q-DEC, Name: Test, dtype: float64

In [6]:
s2

2017-03-31         NaN
2017-06-30   -2.067938
2017-09-30    1.362331
2017-12-31   -0.121087
2018-03-31   -0.866438
2018-06-30    0.502476
2018-09-30    1.985192
2018-12-31   -1.916169
2019-03-31    2.444215
2019-06-30   -1.247837
2019-09-30    1.175958
2019-12-31   -2.585296
2020-03-31    0.348136
2020-06-30   -0.996875
2020-09-30    1.096812
2020-12-31   -0.184987
Freq: Q-DEC, Name: Test, dtype: float64

In [7]:
s3

2017-03-31    0.361663
2017-06-30   -1.706275
2017-09-30   -0.343944
2017-12-31   -0.465031
2018-03-31   -1.331470
2018-06-30   -0.828993
2018-09-30    1.156199
2018-12-31   -0.759970
2019-03-31    1.684245
2019-06-30    0.436408
2019-09-30    1.612366
2019-12-31   -0.972930
2020-03-31   -0.624794
2020-06-30   -1.621668
2020-09-30   -0.524856
2020-12-31   -0.709843
Freq: Q-DEC, dtype: float64

In [8]:
s4

2017-03-31         NaN
2017-06-30   -5.717864
2017-09-30   -0.798424
2017-12-31    0.352054
2018-03-31    1.863183
2018-06-30   -0.377385
2018-09-30   -2.394702
2018-12-31   -1.657300
2019-03-31   -3.216200
2019-06-30   -0.740888
2019-09-30    2.694628
2019-12-31   -1.603417
2020-03-31   -0.357822
2020-06-30    1.595526
2020-09-30   -0.676348
2020-12-31    0.352452
Freq: Q-DEC, Name: Test, dtype: float64

In [9]:
s5

2017-03-31    0.361663
2017-06-30   -1.706275
2017-09-30   -0.343944
2017-12-31   -0.465031
2018-03-31   -1.331470
2018-06-30   -0.828993
2018-09-30    1.156199
2018-12-31   -0.759970
2019-03-31    1.684245
2019-06-30    0.436408
2019-09-30    1.612366
2019-12-31   -0.972930
2020-03-31   -0.624794
2020-06-30   -1.621668
2020-09-30   -0.524856
2020-12-31   -0.709843
Freq: Q-DEC, dtype: float64

In [10]:
s6

2017-03-31    inf
2017-06-30   -inf
2017-09-30   -inf
2017-12-31   -inf
2018-03-31   -inf
2018-06-30   -inf
2018-09-30    inf
2018-12-31   -inf
2019-03-31    inf
2019-06-30    inf
2019-09-30    inf
2019-12-31   -inf
2020-03-31   -inf
2020-06-30   -inf
2020-09-30   -inf
2020-12-31   -inf
Freq: Q-DEC, Name: Test, dtype: float64

In [11]:
s1

2017-03-31    0.361663
2017-06-30   -1.706275
2017-09-30   -0.343944
2017-12-31   -0.465031
2018-03-31   -1.331470
2018-06-30   -0.828993
2018-09-30    1.156199
2018-12-31   -0.759970
2019-03-31    1.684245
2019-06-30    0.436408
2019-09-30    1.612366
2019-12-31   -0.972930
2020-03-31   -0.624794
2020-06-30   -1.621668
2020-09-30   -0.524856
2020-12-31   -0.709843
Freq: Q-DEC, Name: Test, dtype: float64

In [12]:
dates=pd.date_range('2017',periods = 16, freq = 'Q')
sigma, mu = 1, 0
dataseries=pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],index=dates)
df=pd.DataFrame({'Test':dataseries})
df['Type'] = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
df

,Test,Type
2017-03-31,1,0
2017-06-30,2,0
2017-09-30,3,0
2017-12-31,4,0
2018-03-31,5,1
2018-06-30,6,1
2018-09-30,7,1
2018-12-31,8,1
2019-03-31,9,1
2019-06-30,10,1


In [13]:
# Q4 base:
series = df['Test']
s = series.resample('A-DEC').sum()[0:]
s

2017-12-31    10
2018-12-31    26
2019-12-31    42
2020-12-31    58
Freq: A-DEC, Name: Test, dtype: int64

In [14]:
# Q4 base:
series = df['Test']
s = series.resample('A-DEC').mean()[0:]
s

2017-12-31     2.5
2018-12-31     6.5
2019-12-31    10.5
2020-12-31    14.5
Freq: A-DEC, Name: Test, dtype: float64

In [15]:
# Q1 base:
series = df['Test']
s = series.resample('A-MAR').sum()[1:-1]
s

2018-03-31    14
2019-03-31    30
2020-03-31    46
Freq: A-MAR, Name: Test, dtype: int64

In [16]:
# Q4 base:
s0 = df['Test']
dates = series.index
s1  = series.resample('A-DEC').mean()
s1

2017-12-31     2.5
2018-12-31     6.5
2019-12-31    10.5
2020-12-31    14.5
Freq: A-DEC, Name: Test, dtype: float64

In [17]:
s2 = s1.resample('Q').bfill()
s2 = s2[s2.index.year >= 2018]
s2

2018-03-31     6.5
2018-06-30     6.5
2018-09-30     6.5
2018-12-31     6.5
2019-03-31    10.5
2019-06-30    10.5
2019-09-30    10.5
2019-12-31    10.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Freq: Q-DEC, Name: Test, dtype: float64